# TSA Chapter 5: Bitcoin Rolling VaR

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_btc_var/TSA_ch5_btc_var.ipynb)

GARCH-based rolling Value-at-Risk for Bitcoin.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)
import yfinance as yf
from arch import arch_model

In [ ]:
import yfinance as yf

# Download Bitcoin data
btc = yf.download('BTC-USD', start='2019-01-01', end='2025-12-31', progress=False)
if isinstance(btc.columns, pd.MultiIndex):
    btc.columns = btc.columns.get_level_values(0)
btc['Return'] = btc['Close'].pct_change() * 100
btc.dropna(inplace=True)
btc_returns = btc['Return']
print(f"Bitcoin: {len(btc)} observations")

In [ ]:
# Rolling GARCH VaR for Bitcoin
returns = btc_returns
test_period = returns.iloc[-500:]
var_levels = []
for i in range(len(test_period)):
    start = max(0, len(returns) - 500 + i - 250)
    end = len(returns) - 500 + i
    window = returns.iloc[start:end]
    mod = arch_model(window, vol='Garch', p=1, q=1, mean='Constant', dist='normal')
    r = mod.fit(disp='off', update_freq=0)
    fcast = r.forecast(horizon=1)
    sigma1 = np.sqrt(fcast.variance.values[-1, 0])
    var_levels.append(-2.326 * sigma1 / 100)
    if (i+1) % 100 == 0: print(f"  {i+1}/{len(test_period)}")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(test_period.index, test_period.values / 100, color=BLUE, lw=0.5, alpha=0.7, label='Daily Returns')
ax.plot(test_period.index, var_levels, color=RED, lw=1.5, label='GARCH VaR 1%')
violations = (test_period.values / 100) < np.array(var_levels)
if violations.any():
    ax.scatter(test_period.index[violations], (test_period.values / 100)[violations],
               color=RED, s=30, zorder=5, marker='o', edgecolor='white', lw=0.5,
               label=f'Violations ({violations.sum()})')
ax.axhline(0, color=GRAY, lw=0.5)
ax.set_xlabel('Date'); ax.set_ylabel('Return')
ax.set_title('Bitcoin: GARCH-Based Rolling VaR 1%', fontweight='bold')
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
legend_bottom(ax, ncol=3, y=-0.18)
fig.tight_layout(); fig.subplots_adjust(bottom=0.20)
save_chart(fig, 'garch_btc_rolling_var'); plt.show()